# Armory Evaluation Declarations, Composition, and Modification

There are data declarations in charmory.blocks that recapitulate standard evaluations
from the armory package. 

In [1]:
import charmory.blocks.cifar10

baseline = charmory.blocks.cifar10.baseline

/home/msw/.virtualenvs/cdao/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The `baseline` evaluation is a composite dataclass with some metadata fields
describing the evaluation:

In [15]:
from pprint import pprint
pprint(f"{type(baseline)=}")
pprint(f"{baseline.name=}\n{baseline.description=}\n{baseline.author=}")

type(baseline)=<class 'charmory.evaluation.Evaluation'>
baseline.name='cifar_baseline'
baseline.description='Baseline cifar10 image classification'
baseline.author='msw@example.com'


The charmory `Evaluation` class was called "Experiment" in prior versions of the
JATIC Armory library, but was renamed to avoid confusion with the MLflow conception
of Experiment, which is a collection of runs.

Along with the metadata, an `Evaluation` contains some required components. The
`dataset` is a `Dataset` object, which specifies an Armory dataset and a pair
of necessary parameters:

In [14]:
pprint(f"{baseline.dataset=}")

('baseline.dataset=Dataset(function=<function cifar10 at 0x7f8a49d153f0>, '
 "framework='numpy', batch_size=64)")


 
the `function` parameter is shown first while details come afterwards; this allows quick
visibility of "this is a cifar10 dataset" from the `__str__` representation of the
object. An `Evaluation` also requires a `Model` and `Scenario`

In [16]:
pprint(f"{baseline.model=}")
pprint(f"{baseline.scenario=}")

('baseline.model=Model(function=<function get_art_model at 0x7f8abb59b880>, '
 'model_kwargs={}, wrapper_kwargs={}, weights_file=None, fit=True, '
 "fit_kwargs={'nb_epochs': 20})")
('baseline.scenario=Scenario(function=<class '
 "'armory.scenarios.image_classification.ImageClassificationTask'>, kwargs={})")


Here we are using a prefab ART model and the standard Armory Image Classification Task
scenario.

Because this is a "baseline" evaluation, it includes no defense, but does use a PGD
attack to calculate adversarial results.

The optional `Metric` field tells Armory that we want to record additional metrics for
this evaluation. In this case, we are interested in the accuracy of the model on
adversarial examples.

In [17]:
pprint(f"{baseline.attack=}")
pprint(f"{baseline.defense=}")
pprint(f"{baseline.metric=}")

('baseline.attack=Attack(function=<class '
 "'art.attacks.evasion.projected_gradient_descent.projected_gradient_descent.ProjectedGradientDescent'>, "
 "kwargs={'batch_size': 1, 'eps': 0.031, 'eps_step': 0.007, 'max_iter': 20, "
 "'num_random_init': 1, 'random_eps': False, 'targeted': False, 'verbose': "
 "False}, knowledge='white', use_label=True, type=None)")
'baseline.defense=None'
("baseline.metric=Metric(profiler_type='basic', "
 "supported_metrics=['accuracy'], perturbation=['linf'], "
 "task=['categorical_accuracy'], means=True, record_metric_per_sample=False)")


The `charmory.blocks` module is intended as a convenient parts cabinet that allows
users to quickly assemble evaluations using standard components. They behave as standard
Python objects, so you can alter the canned definitions:

In [6]:
baseline.metric = None

If you'd prefer to not have additional tracking. Also, the sub-components of an evaluation
are themselves objects, to be composed at user discretion:

In [7]:
print(f"{charmory.blocks.cifar10.metric=}")

# let's put the metric back into baseline
baseline.metric = charmory.blocks.cifar10.metric

charmory.blocks.cifar10.metric=Metric(profiler_type='basic', supported_metrics=['accuracy'], perturbation=['linf'], task=['categorical_accuracy'], means=True, record_metric_per_sample=False)


Instantiation of the `Engine` class using the `Evaluation` object in `baseline` is
straightforward:

In [8]:
import charmory.engine
engine = charmory.engine.Engine(baseline)

2023-06-26 14:33:16  3s INFO     art.estimators.classification.pytorch:get_layers:1196 Inferred 4 hidden layers on PyTorch classifier.
Reading armory config file
Reading armory config file


Epoch 20/20: 100%|██████████| 781/781 [00:02<00:00, 328.25it/s]

Reading armory config file


In [9]:
result = engine.run()

Evaluation:   0%|          | 0/157 [00:00<?, ?it/s]

2023-06-26 14:34:07 54s INFO     art.attacks.evasion.projected_gradient_descent.projected_gradient_descent:generate:199 Creating adversarial samples.
2023-06-26 14:34:10 57s INFO     art.attacks.evasion.projected_gradient_descent.projected_gradient_descent_pytorch:generate:237 Success rate of attack: 70.31%


Evaluation:   1%|          | 1/157 [00:03<08:15,  3.18s/it]

2023-06-26 14:34:10 57s INFO     art.attacks.evasion.projected_gradient_descent.projected_gradient_descent:generate:199 Creating adversarial samples.
2023-06-26 14:34:14  1m INFO     art.attacks.evasion.projected_gradient_descent.projected_gradient_descent_pytorch:generate:237 Success rate of attack: 75.00%


Evaluation:   1%|▏         | 2/157 [00:06<08:28,  3.28s/it]

...

2023-06-26 14:43:09 9m56s INFO     art.attacks.evasion.projected_gradient_descent.projected_gradient_descent:generate:199 Creating adversarial samples.
2023-06-26 14:43:13 10m INFO     art.attacks.evasion.projected_gradient_descent.projected_gradient_descent_pytorch:generate:237 Success rate of attack: 70.31%


Evaluation:  99%|█████████▉| 156/157 [09:05<00:03,  3.42s/it]

2023-06-26 14:43:13 10m INFO     art.attacks.evasion.projected_gradient_descent.projected_gradient_descent:generate:199 Creating adversarial samples.
2023-06-26 14:43:14 10m INFO     art.attacks.evasion.projected_gradient_descent.projected_gradient_descent_pytorch:generate:237 Success rate of attack: 68.75%


Evaluation: 100%|██████████| 157/157 [09:06<00:00,  3.48s/it]

2023-06-26 14:43:14 10m METRIC   armory.instrument.instrument:_write:743 benign_mean_categorical_accuracy on benign examples w.r.t. ground truth labels: 0.521
2023-06-26 14:43:14 10m METRIC   armory.instrument.instrument:_write:743 adversarial_mean_categorical_accuracy on adversarial examples w.r.t. ground truth labels: 0.0009


In [13]:
from pprint import pprint
pprint(result)

{'armory_version': '23.4.0.post35+gb150c3ac',
 'evaluation': Evaluation(name='cifar_baseline',
                          description='Baseline cifar10 image classification',
                          author='msw@example.com',
                          model=Model(function=<function get_art_model at 0x7f8abb59b880>,
                                      model_kwargs={},
                                      wrapper_kwargs={},
                                      weights_file=None,
                                      fit=True,
                                      fit_kwargs={'nb_epochs': 20}),
                          scenario=Scenario(function=<class 'armory.scenarios.image_classification.ImageClassificationTask'>,
                                            kwargs={}),
                          dataset=Dataset(function=<function cifar10 at 0x7f8a49d153f0>,
                                          framework='numpy',
                                          batch_size=64),
       

# Recap

There is a bunch of explanation and debug prints in this notebook, but the
working code used is quite short:

In [11]:
import charmory.blocks.cifar10
import charmory.engine

baseline = charmory.blocks.cifar10.baseline
engine = charmory.engine.Engine(baseline)
result = engine.run()

2023-06-26 14:43:14 10m INFO     art.estimators.classification.pytorch:get_layers:1196 Inferred 4 hidden layers on PyTorch classifier.
Reading armory config file
Reading armory config file


Epoch 20/20: 100%|██████████| 781/781 [00:02<00:00, 306.54it/s]


Reading armory config file


Evaluation:   0%|          | 0/157 [00:00<?, ?it/s]

2023-06-26 14:44:05 10m52s INFO     art.attacks.evasion.projected_gradient_descent.projected_gradient_descent:generate:199 Creating adversarial samples.
2023-06-26 14:44:08 10m55s INFO     art.attacks.evasion.projected_gradient_descent.projected_gradient_descent_pytorch:generate:237 Success rate of attack: 81.25%


...

2023-06-26 14:48:27 15m14s INFO     art.attacks.evasion.projected_gradient_descent.projected_gradient_descent:generate:199 Creating adversarial samples.
2023-06-26 14:48:31 15m18s INFO     art.attacks.evasion.projected_gradient_descent.projected_gradient_descent_pytorch:generate:237 Success rate of attack: 68.75%


Evaluation:  50%|█████     | 79/157 [04:25<04:16,  3.29s/it]

2023-06-26 14:48:31 15m18s INFO     art.attacks.evasion.projected_gradient_descent.projected_gradient_descent:generate:199 Creating adversarial samples.


Evaluation:  50%|█████     | 79/157 [04:27<04:24,  3.39s/it]


KeyboardInterrupt: 